In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime
from pyspark.sql import Window
from pyspark.sql.types import *
import sys
import os
app_name = 'selection bu batch100'
# spark = SparkSession.builder.appName(app_name).enableHiveSupport().getOrCreate()
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/usr/bin/python3"

spark = (SparkSession\
         .builder\
         .appName("test")\
         .enableHiveSupport()\
         .config("spark.executor.instances", "100")\
         .config("spark.executor.memory","48g")\
         .config("spark.executor.cores","4")\
         .config("spark.driver.memory","40g")\
         .config("spark.sql.shuffle.partitions","2000")\
         .config("spark.default.parallelism","2000")\
         .config("spark.driver.maxResultSize", "8g")\
         .config("spark.pyspark.python", "/usr/bin/python3")\
         .config("spark.sql.autoBroadcastJoinThreshold","-1")
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.executorEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .config("spark.executorEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .getOrCreate())

spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

import datetime
import sys
import pandas as pd
import os
import numpy as np
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
import pyspark.sql.types as sql_type
import spa_utils

spark.sql("set hive.exec.dynamic.partition=true")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.sql('''set hive.exec.max.dynamic.partitions=200551''')
spark.sql('''set hive.exec.max.dynamic.partitions.pernode=200551''')
params = {'author':'xiaoxiao10'}

In [4]:
import datetime
import sys
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.spatial.distance import cosine
name = locals()

## 读训练模型

In [5]:
second_result3 = spark.sql('''select * from dev.dev_xianzhi_model_data_3_result_zou ''')
second_result3.cache()
second_result3.count()

191383

## 读需要预测的数据及特征(新数据剔除旧数据)

In [4]:
df = spark.sql('''select * from dev.dev_xianzhi_model_data_xiaoxiao_6''')
df.cache()
df.count()

1043972

In [5]:
# 用来预测的预测集必须在45天之前，才能保证数据完整
df_dt = df.select('item_sku_id','dt').distinct()
df_dt = df_dt.withColumn('rank',F.row_number().over(Window.partitionBy('item_sku_id').orderBy(F.col('dt').desc())))
df_dt = df_dt.withColumn('predict_flag',F.when(F.col('rank')<=45,0).otherwise(F.lit(1)))

# 每个sku可以使用模型输出数据集的倒数15天之后的数据，这样才数据泄露，而这些数据在之前由于不完整所以未使用
# 且模型输出数据集中没有sku，不会参与新的预测
# 带着历史最高销量字段
model_dt = second_result3.select('item_sku_id','dt','max_qtty').distinct()
model_dt = model_dt.withColumn('rank',F.row_number().over(Window.partitionBy('item_sku_id').orderBy(F.col('dt').desc())))
model_dt2 = model_dt.filter(F.col('rank')==15).select('item_sku_id','dt','max_qtty').withColumnRenamed('dt','valid_dt')

test_dt = df_dt.filter(F.col('predict_flag')==1).select('item_sku_id','dt')\
               .join(model_dt2,'item_sku_id','inner')

test_dt2 = test_dt.filter(F.col('dt')>=F.col('valid_dt')).select('item_sku_id','dt','max_qtty')

predict_df = df.join(test_dt2,['item_sku_id','dt'],'inner')
predict_df.cache()
predict_df.count()

150

## 使用存储的模型预测

In [6]:
model_record = second_result3.select('item_sku_id','model_type','model','huber_model').distinct()

In [7]:
predict_df = predict_df.join(model_record,'item_sku_id','inner')
predict_df.cache()
predict_df.count()

150

In [8]:
X_SCHEMA_SKU = ['target_price', 'decomposedtrend','cid3trend','target_days_flag']

import pickle
import codecs
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor

SCHEMA_OUTPUT_SKU = StructType([
    StructField("item_sku_id", sql_type.StringType()),
    StructField("dt", sql_type.StringType()),
    StructField("model_type", sql_type.StringType()),
    StructField("model", sql_type.StringType()),
    StructField("huber_model", sql_type.StringType()),
    StructField("target_price", sql_type.FloatType()),
    StructField("target_qtty", sql_type.FloatType()),
    StructField("predict", sql_type.FloatType()),
    StructField("predict2", sql_type.FloatType())
])


def format_result_sku(row):
    return (
        str(row['item_sku_id']),
        str(row['dt']),
        str(row['model_type']),
        str(row['model']),
        str(row['huber_model']),
        float(row['target_price']),
        float(row['target_qtty']),
        float(row['predict']),
        float(row['predict2'])
    )

def yichang(ddf):
    ddf = ddf.reset_index()
    if list(ddf.sort_values('target_price')['index']) != list(ddf.sort_values('target_qtty',ascending=False)['index']):
        return 1
    elif list(ddf.sort_values('target_price')['index']) != list(ddf.sort_values('predict',ascending=False)['index']):
        return 1
    else:
        return 0

def smooth(row):    
    df = pd.DataFrame(list(row[1]),columns=['item_sku_id','dt']+X_SCHEMA_SKU+['model_type','model','huber_model','max_qtty','target_qtty'])
    
    # 正常模型预测输出
    sku_model = pickle.loads(codecs.decode(df.iloc[0]['model'].encode(), 'base64'))
    if df.iloc[0]['model_type'] == 'xgboost':
        x = xgboost.DMatrix(df[X_SCHEMA_SKU].values)
        predict_qtty = np.exp(sku_model.predict(x))-1
        df2 = pd.concat([df,pd.DataFrame({'predict':predict_qtty})],axis=1)
        
    elif df.iloc[0]['model_type'] == 'prophet':
        df['ds'] = pd.to_datetime(df['dt'])
        x = df[['ds'] + ['target_price', 'target_days_flag']].reset_index(drop=True)
        forecast = sku_model.predict(x)
        df2 = df.copy()
        df2['predict'] = forecast['yhat']        
    else:
        x = df[X_SCHEMA_SKU]
        predict_qtty = np.exp(sku_model.predict(x))-1
        df2 = pd.concat([df,pd.DataFrame({'predict':predict_qtty})],axis=1)

    
    # 一天有多个价格的随机森林模型，如果发生异常顺序，使用huber再预测一次
    yichang_dt = df2.groupby('dt').apply(lambda x:yichang(x)).reset_index().rename(columns={0:'dt_flag'})
    yichang_dt = yichang_dt.loc[yichang_dt['dt_flag']==1,'dt']
    
    
    if (df.iloc[0]['model_type'] == 'rf') & (len(yichang_dt)>0):
        huber_model = pickle.loads(codecs.decode(df.iloc[0]['huber_model'].encode(), 'base64'))
        
        predict_qtty2 = huber_model.predict(df2[['target_price']])
        df3 = pd.concat([df2,pd.DataFrame({'predict2':predict_qtty2})],axis=1)
        df3.loc[df3['dt'].isin(list(yichang_dt))==False,'predict2'] = df3.loc[df3['dt'].isin(list(yichang_dt))==False,'predict']
    else:
        df3 = df2.copy()
        df3['predict2'] = df3['predict']

    
    # 销量要小于等于历史最大销量，且大于等于0
    df3.loc[df3['predict2']>df.iloc[0]['max_qtty'],'predict2'] = df.iloc[0]['max_qtty']
    df3.loc[df3['predict2']<0,'predict2'] = 0
    
    return df3[['item_sku_id','dt','model_type','model','huber_model','target_price','target_qtty','predict','predict2']].to_dict(orient= 'records')

In [9]:
result = predict_df.select(['item_sku_id','dt']+X_SCHEMA_SKU+['model_type','model','huber_model','max_qtty','target_qtty'])\
                   .rdd.map(lambda row: ((row['item_sku_id']), row)).groupByKey()\
                   .flatMap(lambda row : smooth(row))

result2 = spark.createDataFrame(result.map(format_result_sku), schema=SCHEMA_OUTPUT_SKU)
result2.cache()
result2.count()

Py4JJavaError: An error occurred while calling o318.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: ShuffleMapStage 26 (count at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: Failed to connect to BJLFRZ-10k-186-232.hadoop.jd.local/172.21.186.232:7337 	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:554) 	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:485) 	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:64) 	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435) 	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441) 	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409) 	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31) 

In [ ]:
result2 = result2.withColumn('percent',F.abs(F.col('target_qtty')-F.col('predict'))/F.when(F.col('target_qtty')==0,F.lit(1)).otherwise(F.col('target_qtty')))\
                 .withColumn('second_percent',F.abs(F.col('target_qtty')-F.col('predict2'))/F.when(F.col('target_qtty')==0,F.lit(1)).otherwise(F.col('target_qtty')))

In [ ]:
result2.write.mode('overwrite').format('orc').saveAsTable('dev.dev_xianzhi_model_data_3_result_zou_test')
spark.sql('''ALTER TABLE dev.dev_xianzhi_model_data_3_result_zou_test set TBLPROPERTIES ('author'='kangwenjia1') ''')

In [ ]:
result2.agg(F.mean('second_percent')).show()

In [ ]:
result2.filter(F.col('target_qtty')>=10).agg(F.mean('second_percent')).show()

In [ ]:
wmape = result2.withColumn('cha',F.abs(F.col('target_qtty')-F.col('predict2')))
sum_qtty = wmape.filter(F.col('target_qtty')>=10)\
                .agg(F.sum('target_qtty')).collect()[0][0]

wmape.filter(F.col('target_qtty')>=10)\
     .agg((F.sum('cha')/sum_qtty).alias('wmape')).show()